In [14]:
%pip install pandas
%pip install matplotlib
%pip install pyarrow
%pip install dask
%pip install toolkit


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [2]:
import sys
import os
# Adicionar o caminho do toolkit localmente
toolkit_path = '/Users/ariel/Documents/hackathon-petro/toolkit'
sys.path.append(toolkit_path)

In [4]:
%pip install -e /Users/ariel/Documents/hackathon-petro/toolkit

Defaulting to user installation because normal site-packages is not writeable
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /Users/ariel/Documents/hackathon-petro/toolkit
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
